<a href="https://colab.research.google.com/github/zahraDehghanian97/LSTM_sequence_prediction/blob/master/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Mount google drive**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **prerequisit**

In [3]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.utils import Sequence

# **load data**

In [6]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/predict_next_game/train_Copy.csv')
data = data.dropna()
X = np.array([])
y = np.array([])
for d in data.values:
    X = np.append(X, np.array(d[1].split(" ")))
    y = np.append(y, d[2])
print(X.shape)

(4688,)


# **LSTM Model**

In [9]:
# constant
num_class = 7736
dimension = 1
lstm_units = 200



class MyBatchGenerator(Sequence):
    def __init__(self, X, y, batch_size=1, shuffle=True):
        self.X = X
        self.y = y
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.y) / self.batch_size))

    def __getitem__(self, index):
        return self.__data_generation(index)

    def on_epoch_end(self):
        self.indexes = np.arange(len(self.y))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, index):
        Xb = np.empty((self.batch_size, *X[index].shape))
        yb = np.empty((self.batch_size, *y[index].shape))
        for s in range(0, self.batch_size):
            Xb[s] = X[index]
            yb[s] = y[index]
        return Xb, yb


model = Sequential()
model.add(LSTM(lstm_units, return_sequences=True, input_shape=(None,dimennsion)))
model.add(Dropout(rate=0.3))
model.add(LSTM(lstm_units, return_sequences=True))
model.add(Dropout(rate=0.3))
model.add(LSTM(lstm_units, return_sequences=True))
model.add(Dropout(rate=0.3))
model.add(LSTM(lstm_units, return_sequences=True))
model.add(Dropout(rate=0.3))
model.add(Dense(units=2 * num_class, activation='relu'))
model.add(Dense(units=num_class))
model.compile(optimizer='rmsprop', loss=SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
print(model.summary())
model.fit_generator(MyBatchGenerator(X, y, batch_size=1), epochs=10)


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, None, 200)         161600    
_________________________________________________________________
dropout_4 (Dropout)          (None, None, 200)         0         
_________________________________________________________________
lstm_6 (LSTM)                (None, None, 200)         320800    
_________________________________________________________________
dropout_5 (Dropout)          (None, None, 200)         0         
_________________________________________________________________
lstm_7 (LSTM)                (None, None, 200)         320800    
_________________________________________________________________
dropout_6 (Dropout)          (None, None, 200)         0         
_________________________________________________________________
lstm_8 (LSTM)                (None, None, 200)        

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


ValueError: ignored